# CS-6580 Lecture 20 - Introduction to Neural Networks

**Dylan Zwick**

*Weber State University*

The progress in and application of neural networks is arguably the single most important thing happening in the world today.

Yeah, I know that's a bold statement - but it's defensible. In the last decade there has been an explosion in the depth and difficulty of problems neural networks have been able to tackle, and it looks like further algorithmic advances are unnecessary for this progress to continue. The only thing that's necessary for them to keep getting better is more data, more parameters, and more compute. In other words, it's just a resource question.

As such, neural networks are in an interesting position regarding how they're used by businesses. If you want to build a model for tackling a prediction problem on a relative small amount of data - say the data most companies deal with regarding things like sales, engagement, expenses, etc.. - then a neural network probably won't be the best option. However, if you've got a big problem, and significant resources for tackling it, a neural network can produce results that are, in some cases, absolutely amazing. The most recent, and significant, advance along these lines have been regarding *large language models* (LLMs), and in particular GPT-3, GPT-4, and the ChatGPT interface. We'll probably see a lot more of this in the next 5 years, and their potential impact on the world could be huge. Like, really huge.

Today, we're going to talk about the basic idea behind a neural network, and see how one can work in action on a very famous problem - classifying hand written digits. We won't get into too much depth on either the mathematics behind how it works or the specifics of the code. That will come next week.

**The Single-Layer Neural Network (Perceptron) and Logistic Regression**

The simplest type of neural network - a neural network without any internal (or "hidden") layers, is called a "perceptron", represented below:

![Perceptron](Perceptron.png)

The idea here is that it takes as its input the values from our input variables $X_{1},X_{2}, \ldots, X_{n}$, multiplies them by their appropriate weights, adds these multiplied weights together, feeds these to an activation function, and then to a unit step function.

This might look complicated, but if the activation function is a sigmoid (which it frequently is) then this is just logistic regression.

![Sigmoid](Sigmoid.png)

How do we figure out what our weights are (or should be) with logistic regression? We can't minimize our loss function exactly (like we do with regular regression), and so we look for a minimum using *gradient descent*. Exactly the same idea applies here. The weights are updated based upon whether the prediction is correct, and this is repeated multiple times in order to tune the model. That's it. That's all there is to it. A perceptron with a sigmoid activation function is just logistic regression.

Perceptrons were put forth as a very early model for modeling human cognition in the 1940s and 1950s, and were the subject of the famous 1969 book "Perceptrons" by Marvin Minsky and Seymour Papert, in which they proved, among many other things, that it's impossible for a perceptron to learn the XOR function. In other words, while a perceptron may have its place, it's rather limited.

How can we expand the power of the perceptron? We go deep.

**Multi-Layer Neural Network a.k.a. Deep Learning**

A multi-layer neural network (which essentally essentially everything that's actually called a neural network) is just a bunch of perceptrons (or whatever activation function you want to use) meshed together.

![Multi-Layer Neural Network](Multi-Layer.png)

The number of layers and size of each layer is a hyperparameter that is set before the model trains, and then during model training, what you do is update the weights. How these weights are updated is through something called "back propagation", which means the errors from the output layer are fed back to the previous layer and the weights are adjusted, the error for the previous layer is then fed back to the layer before it, and so on. It turns out that these steps can be handled sequentially in this manner, so that the entire neural network doesn't need to be modified at once. This significantly decreases the complexity of an update (a step in gradient descent), and the reason you're able to do this is, essentially, just the chain rule from calculus.

**Classifying Handwritten Digits**

One of the classic examples in machine learning is figuring out how to correctly classify a handwritten digit. It's very difficult to say exactly what constituties a 7, but I think we'd all agree we (almost always) know one when we see it. For example, all of these are the different ways of writing the number 7:

![Sevens](Sevens.png)

Today, we're going to build a neural network to learn how to distinguish between handwritten digits.

The dataset we'll use is taken from the publicly available MNIST (Modified National Institute of Standards and Technology) dataset which you can find here:http://yann.lecun.com/exdb/mnist/ 

It consists of the following parts:

* Training set images: train-images-idx3-ubyte.gz (9.9 MB, 47 MB unzipped, 60,000 examples)
* Training set labels: train-labels-idx1-ubyte.gz (29 KB, 60 KB unzipped, 60,000 labels)
* Test set images: t10k-images-idx3-ubyte.gz (1.6 MB, 7.8 MB, 10,000 examples)
* Test set labels: t10k-labels-idx1-ubyte.gz (5 KB, 10 KB unzipped, 10,000 labels)

Today we will only be working with a subset of MNIST, and so we only need to download the training set images and training set labels.

First, we'll unzip the files.

In [ ]:
# This code unzips the MNIST dataset.

import sys
import gzip
import shutil
import os

zipped_mnist = [f for f in os.listdir() if f.endswith('ubyte.gz')]
for z in zipped_mnist:
    with gzip.GzipFile(z, mode='rb') as decompressed, open(z[:-3], 'wb') as outfile:
        outfile.write(decompressed.read())

Next, we'll create a function for loading and processing the training data. Don't worry about the details here.

In [ ]:
import os
import struct
import numpy as np
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

OK, now that we have our data, let's use the load function we created above to create a training set and a test set.

In [ ]:
X_train, y_train = load_mnist('', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

In [ ]:
X_test, y_test = load_mnist('', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

We can use matplotlib to visualize the first digit of each class in the training data.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(10):
    img = X_train[y_train == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

And, with a little bet more data cleaning / preparation (again, don't worry about the details right now), we have our training data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

np.savez_compressed('mnist_scaled.npz', 
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_test,
                    y_test=y_test)

mnist = np.load('mnist_scaled.npz')
mnist.files

X_train, y_train, X_test, y_test = [mnist[f] for f in ['X_train', 'y_train', 
                                    'X_test', 'y_test']]

del mnist

X_train.shape

Now, let's build a multi-layer neural network for classifying this data. Please note this is a lot, and we'll step through the sections in class next time.

In [ ]:
class NeuralNetMLP(object):
    """ Feedforward neural network / Multi-layer perceptron classifier.

    Parameters
    ------------
    n_hidden : int (default: 30)
        Number of hidden units.
    l2 : float (default: 0.)
        Lambda value for L2-regularization.
        No regularization if l2=0. (default)
    epochs : int (default: 100)
        Number of passes over the training set.
    eta : float (default: 0.001)
        Learning rate.
    shuffle : bool (default: True)
        Shuffles training data every epoch if True to prevent circles.
    minibatch_size : int (default: 1)
        Number of training examples per minibatch.
    seed : int (default: None)
        Random seed for initializing weights and shuffling.

    Attributes
    -----------
    eval_ : dict
      Dictionary collecting the cost, training accuracy,
      and validation accuracy for each epoch during training.

    """
    def __init__(self, n_hidden=30,
                 l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):

        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size

    def _onehot(self, y, n_classes):
        """Encode labels into one-hot representation

        Parameters
        ------------
        y : array, shape = [n_examples]
            Target values.
        n_classes : int
            Number of classes

        Returns
        -----------
        onehot : array, shape = (n_examples, n_labels)

        """
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T

    def _sigmoid(self, z):
        """Compute logistic function (sigmoid)"""
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))

    def _forward(self, X):
        """Compute forward propagation step"""

        # step 1: net input of hidden layer
        # [n_examples, n_features] dot [n_features, n_hidden]
        # -> [n_examples, n_hidden]
        z_h = np.dot(X, self.w_h) + self.b_h

        # step 2: activation of hidden layer
        a_h = self._sigmoid(z_h)

        # step 3: net input of output layer
        # [n_examples, n_hidden] dot [n_hidden, n_classlabels]
        # -> [n_examples, n_classlabels]

        z_out = np.dot(a_h, self.w_out) + self.b_out

        # step 4: activation output layer
        a_out = self._sigmoid(z_out)

        return z_h, a_h, z_out, a_out

    def _compute_cost(self, y_enc, output):
        """Compute cost function.

        Parameters
        ----------
        y_enc : array, shape = (n_examples, n_labels)
            one-hot encoded class labels.
        output : array, shape = [n_examples, n_output_units]
            Activation of the output layer (forward propagation)

        Returns
        ---------
        cost : float
            Regularized cost

        """
        L2_term = (self.l2 *
                   (np.sum(self.w_h ** 2.) +
                    np.sum(self.w_out ** 2.)))

        term1 = -y_enc * (np.log(output))
        term2 = (1. - y_enc) * np.log(1. - output)
        cost = np.sum(term1 - term2) + L2_term
        
        # If you are applying this cost function to other
        # datasets where activation
        # values maybe become more extreme (closer to zero or 1)
        # you may encounter "ZeroDivisionError"s due to numerical
        # instabilities in Python & NumPy for the current implementation.
        # I.e., the code tries to evaluate log(0), which is undefined.
        # To address this issue, you could add a small constant to the
        # activation values that are passed to the log function.
        #
        # For example:
        #
        # term1 = -y_enc * (np.log(output + 1e-5))
        # term2 = (1. - y_enc) * np.log(1. - output + 1e-5)
        
        return cost

    def predict(self, X):
        """Predict class labels

        Parameters
        -----------
        X : array, shape = [n_examples, n_features]
            Input layer with original features.

        Returns:
        ----------
        y_pred : array, shape = [n_examples]
            Predicted class labels.

        """
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_valid, y_valid):
        """ Learn weights from training data.

        Parameters
        -----------
        X_train : array, shape = [n_examples, n_features]
            Input layer with original features.
        y_train : array, shape = [n_examples]
            Target class labels.
        X_valid : array, shape = [n_examples, n_features]
            Sample features for validation during training
        y_valid : array, shape = [n_examples]
            Sample labels for validation during training

        Returns:
        ----------
        self

        """
        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]

        ########################
        # Weight initialization
        ########################

        # weights for input -> hidden
        self.b_h = np.zeros(self.n_hidden)
        self.w_h = self.random.normal(loc=0.0, scale=0.1,
                                      size=(n_features, self.n_hidden))

        # weights for hidden -> output
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0, scale=0.1,
                                        size=(self.n_hidden, n_output))

        epoch_strlen = len(str(self.epochs))  # for progress formatting
        self.eval_ = {'cost': [], 'train_acc': [], 'valid_acc': []}

        y_train_enc = self._onehot(y_train, n_output)

        # iterate over training epochs
        for i in range(self.epochs):

            # iterate over minibatches
            indices = np.arange(X_train.shape[0])

            if self.shuffle:
                self.random.shuffle(indices)

            for start_idx in range(0, indices.shape[0] - self.minibatch_size +
                                   1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]

                # forward propagation
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])

                ##################
                # Backpropagation
                ##################

                # [n_examples, n_classlabels]
                delta_out = a_out - y_train_enc[batch_idx]

                # [n_examples, n_hidden]
                sigmoid_derivative_h = a_h * (1. - a_h)

                # [n_examples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_examples, n_hidden]
                delta_h = (np.dot(delta_out, self.w_out.T) *
                           sigmoid_derivative_h)

                # [n_features, n_examples] dot [n_examples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, delta_h)
                grad_b_h = np.sum(delta_h, axis=0)

                # [n_hidden, n_examples] dot [n_examples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, delta_out)
                grad_b_out = np.sum(delta_out, axis=0)

                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2*self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h

                delta_w_out = (grad_w_out + self.l2*self.w_out)
                delta_b_out = grad_b_out  # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out

            #############
            # Evaluation
            #############

            # Evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._compute_cost(y_enc=y_train_enc,
                                      output=a_out)

            y_train_pred = self.predict(X_train)
            y_valid_pred = self.predict(X_valid)

            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) /
                         X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(np.float) /
                         X_valid.shape[0])

            sys.stderr.write('\r%0*d/%d | Cost: %.2f '
                             '| Train/Valid Acc.: %.2f%%/%.2f%% ' %
                             (epoch_strlen, i+1, self.epochs, cost,
                              train_acc*100, valid_acc*100))
            sys.stderr.flush()

            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['valid_acc'].append(valid_acc)

        return self

OK! Now we've got our neural network. Please note that, of course, Python has built in neural network libraries, so generally speaking wo don't need to build a neural network from scratch. 

Alright, let's set the number of training iterations we want to use, a.k.a "epochs".

In [ ]:
n_epochs = 200

Finally, we'll run our neural network on our training data. *Warning* - This can take a minute.

In [ ]:
nn = NeuralNetMLP(n_hidden=100, 
                  l2=0.01, 
                  epochs=n_epochs, 
                  eta=0.0005,
                  minibatch_size=100, 
                  shuffle=True,
                  seed=1)

nn.fit(X_train=X_train[:55000], 
       y_train=y_train[:55000],
       X_valid=X_train[55000:],
       y_valid=y_train[55000:])

We can graph the performance of this neural network over the 200 iterations:

In [ ]:
plt.plot(range(nn.epochs), nn.eval_['cost'])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.show()

We can also look at how the model performed on the training dataset compared with the validation dataset.

In [ ]:
plt.plot(range(nn.epochs), nn.eval_['train_acc'], 
         label='Training')
plt.plot(range(nn.epochs), nn.eval_['valid_acc'], 
         label='Validation', linestyle='--')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='lower right')
plt.show()

So, it looks like it legitimately gets better bor about the first 50 epochs or so, but then it just gets better at predicting the training data, although it doesn't look like that introduces significant overfitting issues.

Alright, now for the final test we can look at how it does on the holdout test data.

In [ ]:
y_test_pred = nn.predict(X_test)
acc = (np.sum(y_test == y_test_pred)
       .astype(np.float) / X_test.shape[0])

print('Test accuracy: %.2f%%' % (acc * 100))

Pretty good! Are you curious about the ones that it missed? Well, we can check those out. Let's take a look at the first 25.

In [ ]:
miscl_img = X_test[y_test != y_test_pred][:25]
correct_lab = y_test[y_test != y_test_pred][:25]
miscl_lab = y_test_pred[y_test != y_test_pred][:25]

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = miscl_img[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title('%d) t: %d p: %d' % (i+1, correct_lab[i], miscl_lab[i]))

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

Honestly, I can see how it missed some of these. I'm not entirely sure whether 19 is a 4 or a 9, whether 21 is a 2 or a 7, or whether 23 is an 8 or a 9. So, I think the model is getting pretty close to human performance, which will not be perfect. In fact, the dream of a perfect model probably isn't achieveably under any cincumstance, because of inconsistencies in how the data is labeled.